Looks like there are 4 sections, 'Food Manufacturer', 'Restaurant/Fast Food', 'Supermarket' and 'Other'

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as nps
import unidecode
import math

Only 4 different links for the different kinds of foods, and should all be scraped in the same format

In [2]:
fd_manufac_url = 'https://www.fatsecret.com/Default.aspx?pa=brands&t=1'
rest_url = 'https://www.fatsecret.com/Default.aspx?pa=brands&t=2'
mrkt_url = 'https://www.fatsecret.com/Default.aspx?pa=brands&t=3'
other_url = 'https://www.fatsecret.com/Default.aspx?pa=brands&t=4'

In [3]:
fd_page = requests.get(fd_manufac_url)
fd_soup = BeautifulSoup(fd_page.text, 'html.parser')

Starting From the Bottom - Getting All Nutrition

In [7]:
prod_url = 'https://www.fatsecret.com/calories-nutrition/a-la-carte/chicken-cordon-bleu'
prod_page = requests.get(prod_url)
prod_soup = BeautifulSoup(prod_page.text, 'html.parser')

In [75]:
#fact_panel = prod_soup.find('td', attrs={'class':'factPanel', 'width':'255'})
fact_panel = prod_soup.find('div', attrs={'class':'nutpanel'})

In [76]:
rows = fact_panel.findAll('tr')

In [85]:
# Getting serving size
for row in rows:
    if row.text.strip().find('Serving Size:') != -1:
        serving_size = row.text.strip().replace('Serving Size:', u'').strip()
        serving_size = unidecode.unidecode(serving_size)
        serving_size = serving_size.replace('\r',u'').strip()
        serving_size = serving_size.replace('\n',u' ').strip()
        serving_size = serving_size.replace('\t',u'').strip()
    elif row.text.strip().find('Calories') != -1:
        calories = row.find('td').text.strip()
        cal_label = row.find('b').text.strip()
        cal_fat_label = row.find('div').text.strip()
        calories = calories.replace(cal_fat_label, u'')
        calories = calories.replace(cal_label, u'').strip()
    elif row.text.strip().find('Total Fat') != -1:
        fat = row.find('td').text.strip()
        fat_label = row.find('td').find('b').text.strip()
        fat = fat.replace(fat_label, u'').strip()
    elif row.text.strip().find('Saturated Fat') != -1:
        sat_fat = row.find('td', attrs={'class':'borderTop label'}).text.strip()
        sat_fat = sat_fat.replace('Saturated Fat', u'').strip()
    elif row.text.strip().find('Cholesterol') != -1:
        cholesterol = row.find('td').text.strip()
        cholesterol_label = row.find('td').find('b').text.strip()
        cholesterol = cholesterol.replace(cholesterol_label, u'').strip()
    elif row.text.strip().find('Sodium') != -1:
        sodium = row.find('td').text.strip()
        sodium_label = row.find('td').find('b').text.strip()
        sodium = sodium.replace(sodium_label, u'').strip()
    elif row.text.strip().find('Total Carbohydrate') != -1:
        carb = row.find('td').text.strip()
        carb_label = row.find('td').find('b').text.strip()
        carb = carb.replace(carb_label, u'').strip()
    elif row.text.strip().find('Dietary Fiber') != -1:
        fiber = row.find('td', attrs={'class':'borderTop label'}).text.strip()
        fiber = fiber.replace('Dietary Fiber', u'').strip()
    elif row.text.strip().find('Sugars') != -1:
        sugar = row.find('td', attrs={'class':'borderTop label'}).text.strip()
        sugar = sugar.replace('Sugars', u'').strip()
    elif row.text.strip().find('Protein') != -1:
        protein = row.find('td').text.strip()
        protein = protein.replace('Protein', u'').strip()
    elif row.text.strip().find('Vitamin A') != -1:
        vit_rows = row.findAll('td', attrs={'width':'50%'})
        vit_a = vit_rows[0].text.strip().replace('Vitamin A',u'').strip()
        vit_c = vit_rows[1].text.strip().replace('Vitamin C', u'').strip()
    elif row.text.strip().find('Calcium') != -1:
        calc_iron_rows = row.findAll('td', attrs={'width':'50%'})
        calcium = calc_iron_rows[0].text.strip().replace('Calcium',u'').strip()
        iron = calc_iron_rows[1].text.strip().replace('Iron', u'').strip()

Next - getting the individual products

In [92]:
product_list_url = 'https://www.fatsecret.com/calories-nutrition/search?q=A+La+Carte'
product_list_pg = requests.get(product_list_url)
product_list_soup = BeautifulSoup(product_list_pg.text, 'html.parser')

In [94]:
results_sec = product_list_soup.find('table', attrs={'class':'generic searchResult'})
rows = results_sec.findAll('tr')

In [20]:
base_url = 'https://www.fatsecret.com'

In [100]:
base_url = 'https://www.fatsecret.com'
for row in rows:
    text_line = row.find('td',attrs={'class':'borderBottom'})
    food = text_line.find('a',attrs={'class':'prominent'}).text.strip()
    food_link = base_url+text_line.find('a',attrs={'class':'prominent'})['href']
    if text_line.find('a',attrs={'class':'brand'}) != None:
        brand = text_line.find('a',attrs={'class':'brand'}).text.strip()
        brand = brand.replace('(',u'').replace(')',u'').strip()
    else:
        brand = np.nan
    print(food)
    print(food_link)
    print(brand)
    

Chicken Cordon Bleu
https://www.fatsecret.com/calories-nutrition/a-la-carte/chicken-cordon-bleu
A La Carte
Irish Lamb Stew
https://www.fatsecret.com/calories-nutrition/a-la-carte/irish-lamb-stew
A La Carte
Herbes De Provence Lamb Shanks
https://www.fatsecret.com/calories-nutrition/a-la-carte/herbes-de-provence-lamb-shanks
A La Carte
Vegetable Medley
https://www.fatsecret.com/calories-nutrition/a-la-carte/vegetable-medley
A La Carte
Stephen Colbert's Americone Dream Vanilla Ice Cream with Fudge Covered Waffle Cone Pieces & a Caramel Swirl (Container)
https://www.fatsecret.com/calories-nutrition/ben-and-jerrys/stephen-colberts-americone-dream-vanilla-ice-cream-with-fudge-covered-waffle-cone-pieces-and-a-caramel-swirl-(container)
Ben & Jerry's
Seven-Layer Salad (Lettuce Salad Made with A Combination Of Onion, Celery, Green Pepper, Peas, Mayonnaise, Cheese, Eggs and/or Bacon)
https://www.fatsecret.com/calories-nutrition/generic/seven-layer-salad-(lettuce-salad-made-with-a-combination-of-on

Parsing all pages for a single brand

In [103]:
search_summary = product_list_soup.find('div',attrs={'class':'searchResultSummary'})
txt = search_summary.text.strip()
total_products = txt[(txt.find('of')+2):txt.find('for')].strip()
total_pages = math.ceil(int(total_products)/10)

In [122]:
range(total_pages)

range(0, 62)

In [126]:
brands_page = requests.get(fd_manufac_url)
brands_soup = BeautifulSoup(brands_page.text,'html.parser')

In [127]:
brand_sec = brounds_soup.find('div', attrs={'class':'leftCellContent'})

In [128]:
brand_rows = brand_sec.findAll('h2')

In [129]:
for row in brand_rows:
    brand_name = row.find('a').text.strip()
    brand_link = base_url+row.find('a')['href']
    print(brand_name)
    print(brand_link)

A La Carte
https://www.fatsecret.com/calories-nutrition/a-la-carte
A La Francaise
https://www.fatsecret.com/calories-nutrition/a-la-francaise
A Southern Season
https://www.fatsecret.com/calories-nutrition/a-southern-season
A Taste of India
https://www.fatsecret.com/calories-nutrition/a-taste-of-india
A Taste of Thai
https://www.fatsecret.com/calories-nutrition/a-taste-of-thai
A&W
https://www.fatsecret.com/calories-nutrition/aandw
A. Vogel
https://www.fatsecret.com/calories-nutrition/a-vogel
A.1. Steak Sauces And Marinades
https://www.fatsecret.com/calories-nutrition/a1-steak-sauces-and-marinades
A.K. Gourmet
https://www.fatsecret.com/calories-nutrition/ak-gourmet
A2
https://www.fatsecret.com/calories-nutrition/a2
Aaron's
https://www.fatsecret.com/calories-nutrition/aarons
Ab Cuts
https://www.fatsecret.com/calories-nutrition/ab-cuts
ABB
https://www.fatsecret.com/calories-nutrition/abb
Abbey Bakery
https://www.fatsecret.com/calories-nutrition/abbey-bakery
Abbotsford Farms
https://www.fat

In [130]:
brand_step_pg =requests.get(brand_link)
brand_stp_soup = BeautifulSoup(brand_step_pg.text, 'html.parser')

In [134]:
full_pg_link = base_url+brand_stp_soup.find('div', attrs={'class':'smallText more', 'align':'right'}).find('a')['href']

In [148]:
n_letter_pages = brands_soup.find('div', attrs={'class':'searchResultSummary'}).text.strip()

In [150]:
n_letter_pages[(n_letter_pages.find('of')+2):len(n_letter_pages)].strip()

'490'

In [156]:
i = 0
char = 'A'
t = 1

In [158]:
base_url+'/Default.aspx?pa=brands&pg='+str(i)+'&f='+char.lower()+'&t='+str(t)

'https://www.fatsecret.com/Default.aspx?pa=brands&pg=0&f=a&t=1'

In [160]:
for i in range(1,5):
    print(i)

1
2
3
4


In [161]:
food_types = {1:'grocery',2:'restaurant',3:'super_market',4:'other'}

In [164]:
food_types[3]

'super_market'

In [4]:
brand_pg = requests.get('https://www.fatsecret.com/Default.aspx?pa=brands&t=3')
brand_soup = BeautifulSoup(brand_pg.text, 'html.parser')

In [14]:
search_url = 'https://www.fatsecret.com/calories-nutrition/search?q='
brand_sec = brand_soup.find('div', attrs={'class':'leftCellContent'})
if brand_sec.find('h2') != None:
    brand_rows = brand_sec.findAll('h2')
    for row in brand_rows:
        brand_name = row.find('a').text.strip()
        brand_url = search_url + brand_name.replace(' ', u'+')
        print(brand_url)

https://www.fatsecret.com/calories-nutrition/search?q=A+Taste+of+Thai
https://www.fatsecret.com/calories-nutrition/search?q=Albertsons
https://www.fatsecret.com/calories-nutrition/search?q=Aldi
https://www.fatsecret.com/calories-nutrition/search?q=All+Stars
https://www.fatsecret.com/calories-nutrition/search?q=All+Whites
https://www.fatsecret.com/calories-nutrition/search?q=Allure
https://www.fatsecret.com/calories-nutrition/search?q=American+Fare
https://www.fatsecret.com/calories-nutrition/search?q=America's+Choice
https://www.fatsecret.com/calories-nutrition/search?q=Amick+Farms
https://www.fatsecret.com/calories-nutrition/search?q=Appetitos
https://www.fatsecret.com/calories-nutrition/search?q=Apple+Bobbers
https://www.fatsecret.com/calories-nutrition/search?q=Appleton
https://www.fatsecret.com/calories-nutrition/search?q=Appleton+Farms
https://www.fatsecret.com/calories-nutrition/search?q=Archer+Farms
https://www.fatsecret.com/calories-nutrition/search?q=Art+Dessert
https://www.fa

In [25]:
brand_pg = requests.get('https://www.fatsecret.com/calories-nutrition/search?q=A+Taste+of+Thai&pg=3')
brand_soup = BeautifulSoup(brand_pg.text, 'html.parser')

In [26]:
results_sec = brand_soup.find('table', attrs={'class':'generic searchResult'})

In [27]:
if results_sec == None:
    rows = product_list_soup.findAll('tr')
else:
    rows = results_sec.findAll('tr')

In [28]:
page_brand = 'A Taste of Thai'

In [38]:
for row in rows:
    if row.find('td', attrs={'class':'borderBottom'}).find('a',attrs={'class':'brand'})!=None:
        text_line = row.find('td',attrs={'class':'borderBottom'})

        brand = text_line.find('a',attrs={'class':'brand'}).text.strip()
        brand = brand.replace('(',u'').replace(')',u'').strip()
        print(brand)
        if brand == page_brand:
            food = text_line.find('a',attrs={'class':'prominent'}).text.strip()
            food_link = base_url+text_line.find('a',attrs={'class':'prominent'})['href']
            print(food)
            print(food_link)

A Taste of Thai
Chicken & Rice Seasoning
https://www.fatsecret.com/calories-nutrition/a-taste-of-thai/chicken-and-rice-seasoning
A Taste of Thai
Coconut Milk Unsweetened/First Pressing
https://www.fatsecret.com/calories-nutrition/a-taste-of-thai/coconut-milk-unsweetened-first-pressing
A Taste of Thai
Toasted Coconut Fortune Cookies
https://www.fatsecret.com/calories-nutrition/a-taste-of-thai/toasted-coconut-fortune-cookies
Ben & Jerry's
Taco Time
Taco Time
Taco Time
Old Dutch
Trader Joe's


In [31]:
row

<tr>
<td class="borderBottom">
<a class="prominent" href="/calories-nutrition/generic/chicken-or-turkey-a-la-king-with-vegetables-(including-carrots-broccoli-and-or-dark-green-leafy-(no-potatoes))-cream-white-or-soup-based-sauce">Chicken or Turkey A La King with Vegetables in Cream, White or Soup-Based Sauce (Including Carrots, Broccoli, and/or Dark-Green Leafy, No Potatoes)</a>
<div class="smallText greyText greyLink">
										
											Per 1 cup - Calories: 465kcal | Fat: 33.38g | Carbs: 16.41g | Protein: 24.41g
										
										<!--
										1153 calories per <a href="#">597g serving of Chicken or Turkey A La King with Vegetables in Cream, White or Soup-Based Sauce (Including Carrots, Broccoli, and/or Dark-Green Leafy, No Potatoes)</a>.
										-->
<br/>Other sizes:
													1 serving - 349kcal
											, 1 Banquet's Cookin' Bag - 247kcal
											, 100 g - 193kcal
											
											, <i>more...</i>
										
										   <a href="/calories-nutrition/generic

In [39]:
brand

"Trader Joe's"

In [40]:
brand.replace(' ',u'+')

"Trader+Joe's"

In [41]:
brand

"Trader Joe's"

In [34]:
text_line = row.find('td',attrs={'class':'borderBottom'})
#brand = text_line.find('a',attrs={'class':'brand'}).text.strip()

In [35]:
text_line

<td class="borderBottom">
<a class="prominent" href="/calories-nutrition/generic/chicken-or-turkey-a-la-king-with-vegetables-(including-carrots-broccoli-and-or-dark-green-leafy-(no-potatoes))-cream-white-or-soup-based-sauce">Chicken or Turkey A La King with Vegetables in Cream, White or Soup-Based Sauce (Including Carrots, Broccoli, and/or Dark-Green Leafy, No Potatoes)</a>
<div class="smallText greyText greyLink">
										
											Per 1 cup - Calories: 465kcal | Fat: 33.38g | Carbs: 16.41g | Protein: 24.41g
										
										<!--
										1153 calories per <a href="#">597g serving of Chicken or Turkey A La King with Vegetables in Cream, White or Soup-Based Sauce (Including Carrots, Broccoli, and/or Dark-Green Leafy, No Potatoes)</a>.
										-->
<br/>Other sizes:
													1 serving - 349kcal
											, 1 Banquet's Cookin' Bag - 247kcal
											, 100 g - 193kcal
											
											, <i>more...</i>
										
										   <a href="/calories-nutrition/generic/chic

In [5]:
good_url = 'https://www.fatsecret.com/calories-nutrition/search?q=A.K.+Gourmet'
bad_url = 'https://www.fatsecret.com/calories-nutrition/search?q=A2'

In [11]:
food_pages_url = requests.get(good_url)
food_pages_soup = BeautifulSoup(food_pages_url.text, 'html.parser')
search_summary = food_pages_soup.find('div',attrs={'class':'searchResultSummary'})

In [12]:
search_summary != None

True